In [14]:
%pip install torch torchvision torchaudio pandas numpy matplotlib


Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import numpy as np
import torch
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd


In [16]:
# load fashion mnist dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
fmnist_train = datasets.FashionMNIST('fashionMNIST', train=True, download=True, transform=transform)

In [17]:
# CNN model
class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, 3, 1)
        self.conv2 = torch.nn.Conv2d(32, 64, 3, 1)
        self.fc1 = torch.nn.Linear(5*5*64, 128)
        self.fc2 = torch.nn.Linear(128, 10)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        # print(x.shape)
        x = x.view(-1, 5*5*64)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# train model
def train(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 1000 == 999:
                print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 1000}')
                running_loss = 0.0

In [18]:
# run training
train_loader = torch.utils.data.DataLoader(fmnist_train, batch_size=4, shuffle=True)
classifier = Classifier()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
train(classifier, train_loader, criterion, optimizer, epochs=3)



[1, 1000] loss: 0.7596919068086426
[1, 2000] loss: 0.49098808356636436
[1, 3000] loss: 0.4371311948732473
[1, 4000] loss: 0.41725688544598233
[1, 5000] loss: 0.39605649235812596
[1, 6000] loss: 0.3875735186637321
[1, 7000] loss: 0.3690632801573593
[1, 8000] loss: 0.36051354347725645
[1, 9000] loss: 0.3483217911435722
[1, 10000] loss: 0.36706697714936615
[1, 11000] loss: 0.328453355159408
[1, 12000] loss: 0.3294036126303781
[1, 13000] loss: 0.3211304859822849
[1, 14000] loss: 0.3121015470894672
[1, 15000] loss: 0.31297392972680066
[2, 1000] loss: 0.2639582327926155
[2, 2000] loss: 0.26170967807772283
[2, 3000] loss: 0.2950111890168289
[2, 4000] loss: 0.2904840947453445
[2, 5000] loss: 0.2650629503286191
[2, 6000] loss: 0.2706037216464465
[2, 7000] loss: 0.3048746287730598
[2, 8000] loss: 0.2683704923162849
[2, 9000] loss: 0.26522221759694514
[2, 10000] loss: 0.26267896112863615
[2, 11000] loss: 0.2604888141651529
[2, 12000] loss: 0.2736029814538443
[2, 13000] loss: 0.2656185702275925
[2

In [26]:
# forward pass on 1 example
for img, lab in train_loader:
    output = model(img)
    break
output

tensor([[  3.9903, -10.9505,  -8.1856,  -6.4368, -12.2612, -21.7233,   1.7030,
         -19.0155, -15.7663, -26.5770],
        [-24.7924, -31.5912, -24.9703, -28.6228, -37.8264,  -6.2844, -21.3845,
         -10.9280, -26.2253,   7.3938],
        [ -6.0053, -17.1533,  -3.3409,  -8.0582,  -0.8078, -39.6625,   5.7848,
         -36.5317, -24.0784, -41.7130],
        [-29.1442, -24.4579, -38.2499, -42.0445, -52.1394, -16.4124, -38.1905,
          -9.5167, -22.1381,  13.1875]], grad_fn=<AddmmBackward0>)

In [22]:
# copy model and cut out the last layer
encoder = Classifier()
encoder.load_state_dict(classifier.state_dict())
encoder.fc2 = torch.nn.Identity()

In [28]:
# forward pass on 1 example
for img, lab in train_loader:
    output = encoder(img)
    break

In [30]:
output[0]

tensor([ 0.0000,  0.0000,  5.4285,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0290,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  4.2115,  0.0000,
         0.0000,  0.0000,  0.0000,  9.5629,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  6.2770,  2.7229,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  6.8298,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000, 10.5283,  0.0000,  9.9873,  0.0000,  0.0000,  0.0000,  5.9944,
         0.0000,  0.0000, 11.0462,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.1624,  0.0000,  0.0000,
         0.0000,  0.0000,  1.9188,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.00

In [39]:
fmnist_train[13][1]

5

In [42]:
def compare(dataset, i1, i2):
    # get 2 random images and their labels
    img1, lab1 = dataset[i1]
    img2, lab2 = dataset[i2]

    # forward pass on 2 images
    output1 = encoder(img1.unsqueeze(0))
    output2 = encoder(img2.unsqueeze(0))

    # calculate cosine similarity
    cos = torch.nn.CosineSimilarity(dim=1)

    return lab1 == lab2, cos(output1, output2)

In [43]:
compare(fmnist_train, 13, 12)

(True, tensor([0.8365], grad_fn=<SumBackward1>))

In [44]:
compare(fmnist_train, 13, 13)

(True, tensor([1.], grad_fn=<SumBackward1>))

In [45]:
compare(fmnist_train, 13, 14)

(False, tensor([0.4311], grad_fn=<SumBackward1>))

In [46]:
compare(fmnist_train, 13, 144)

(False, tensor([0.1742], grad_fn=<SumBackward1>))